In [15]:
import numpy as np
import pandas as pd
import xgboost as xgb
from config import *
from cal_accuracy import *
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [16]:
# 加载模型并使用
loaded_model = xgb.Booster()
loaded_model.load_model("xgboost_model.model")

In [17]:
# test_data_path = r'./input/80_mfcc_features.xlsx'
test_data_path = r'./input/BX_mfcc_features.xlsx'
test_data = pd.read_excel(test_data_path)
X_test = test_data[used_feature_columns].values
y_test = test_data['score']
dtest = xgb.DMatrix(X_test)
y_pred = loaded_model.predict(dtest)

In [18]:
y_pred

array([3.5761707, 4.1629415, 3.9302642, 4.1238728, 4.099669 , 4.114581 ,
       4.4742923, 3.7506554, 3.8299625, 4.033918 , 3.6801188, 3.9301906,
       3.4271986, 3.6250556, 4.028126 , 3.7302024, 3.9363422, 4.2527165,
       4.35148  , 4.1254644], dtype=float32)

In [4]:
contrast = pd.DataFrame()
contrast['true'] = y_test
contrast['predict'] = y_pred
contrast['difference'] = abs(y_pred - y_test)
contrast['file_name'] = test_data['file_name']
contrast

,true,predict,difference,file_name
0,4.575,4.100381,0.474619,ba1-M301-1731.wav
1,5.100,4.708733,0.391267,ba2-M302-0823.wav
2,5.150,3.911658,1.238342,ba3-M303-0743.wav
3,3.150,4.505127,1.355127,ba4-M304-1030.wav
4,5.125,5.133733,0.008733,ba5-M306-0708.wav
...,...,...,...,...
75,5.925,4.608159,1.316841,ns11-N601-1050.wav
76,5.200,5.327938,0.127938,ns12-N602-0859.wav
77,4.250,4.785335,0.535335,ns13-N603-1527.wav
78,5.750,4.824992,0.925008,ns14-N604-1029.wav


In [14]:
threshold = 1
count = (contrast['difference'] < threshold).sum()
acc = count / len(contrast)
acc

0.7375

In [6]:
avg_diff = contrast['difference'].sum() / len(contrast)
# avg_percent = avg_diff / 8
avg_percent = avg_diff / 5
(1 - avg_percent)

0.8549230571985245

In [7]:
r2 = r2_score(contrast['true'], contrast['predict'])
r2

0.27287242869102735

In [8]:
predict_class_list = []
true_class_list = []
for i in range(len(contrast)):
    predict_class, true_class = classify(contrast.predict[i], contrast.true[i])
    predict_class_list.append(predict_class)
    true_class_list.append(true_class)

acc = accuracy(predict_class_list, true_class_list)
acc

0.8375

In [9]:
mse = mean_squared_error(contrast['predict'], contrast['true'])
rmse = np.sqrt(mse)
mae = mean_absolute_error(contrast['predict'], contrast['true'])
mse, rmse, mae

(0.8270024486986138, 0.9093967498834674, 0.7253847140073777)